# Setup 
## Update Data Directory

In [1]:
# Update Data/ directory.
%run ./DownloadData.py

## Imports

In [2]:
import os
import numpy as np
import pandas as pd

## Load DataFrames

In [3]:
df = {}
for data in os.listdir('Data/'):
    if data.endswith('.csv'):
        df[data[:-4]] = pd.read_csv('Data/' + data)
print sorted(df.keys())

['caste', 'caste_grouped', 'caste_percent', 'communicabledisease1', 'communicabledisease1T', 'communicabledisease2', 'communicabledisease2T', 'hdi2011', 'master', 'mentalhealth', 'mentalhealthT', 'poverty1', 'poverty1T', 'poverty2', 'poverty2T', 'sexratio', 'vaccinations', 'vaccinationsT', 'watersanitaion', 'watersanitationT']


# Create DataFrames
## caste_percent
This DataFrame summarizes the percent of the population belonging to each caste by district.

In [4]:
series = []
totalpop = {}
for district in df['caste']['District'].unique():
    caste = df['caste'][(df['caste']['District'] == district)].groupby('Caste/Ethnicity').sum()['In number']
    if district == 'Darchula':
        district = 'Darchaula'
    if district == 'Kavrepalanchok':
        district = 'Kavre'
    series.append((100.0 * (caste / caste.sum())).rename(district))
    totalpop[district] = caste.sum()

df['caste_percent'] = pd.DataFrame(series).fillna(0)
df['caste_percent']['Tot_Pop'] = pd.Series(totalpop).rename('Total_Population')
df['caste_percent'].to_csv('Data/caste_percent.csv')

In [5]:
caste = pd.read_csv('Data/caste.csv')
sg = {'Brahman - Hill':'Hill Brahman', 'Chhetree':'Hill Chhetree', 'Thakuri':'Hill Chhetree', \
      'Dashnami/Sanyasi':'Hill Chhetree', 'Brahman - Tarai':'Madhesi Brahman/Chhetree', \
     'Kayastha':'Madhesi Brahman/Chhetree', 'Rajput':'Madhesi Brahman/Chhetree', 'Amat':'Madhesi Other - OC A', \
     'Badhaee':'Madhesi Other - OC A', 'Baraee':'Madhesi Other - OC A', 'Dev':'Madhesi Other - OC A', \
     'Gaderi/Bhedhar':'Madhesi Other - OC A', 'Hajam/Thakur':'Madhesi Other - OC A', \
     'Haluwai':'Madhesi Other - OC A', 'Kalar':'Madhesi Other - OC A', 'Kalwar':'Madhesi Other - OC A', \
     'Kamar':'Madhesi Other - OC A', 'Kathbaniyan':'Madhesi Other - OC A', 'Koiri/Kushwaha':'Madhesi Other - OC A', \
     'Kurmi':'Madhesi Other - OC A', 'Lohar':'Madhesi Other - OC A', 'Rajbhar':'Madhesi Other - OC A', \
     'Rajdhob':'Madhesi Other - OC A', 'Sonar':'Madhesi Other - OC A', 'Sudhi':'Madhesi Other - OC A', 'Kanu':'Madhesi Other - OC A', \
     'Teli':'Madhesi Other - OC A', 'Yadav':'Madhesi Other - OC A', 'Bin':'Madhesi Other - OC B', \
     'Dhandi':'Madhesi Other - OC B', 'Dhankar/Kharikar':'Madhesi Other - OC B', 'Dhunia':'Madhesi Other - OC B', \
     'Kahar':'Madhesi Other - OC B', 'Kewat':'Madhesi Other - OC B', 'Kori':'Madhesi Other - OC B', \
     'Kumhar':'Madhesi Other - OC B', 'Lodh':'Madhesi Other - OC B', 'Mali':'Madhesi Other - OC B', \
     'Mallaha':'Madhesi Other - OC B', 'Natuwa':'Madhesi Other - OC B', 'Nuniya':'Madhesi Other - OC B', \
     'Nurang':'Madhesi Other - OC B', 'Sarbaria':'Madhesi Other - OC B', 'Badi': 'Hill Dalit', \
     'Damai/Dholi':'Hill Dalit', 'Gaine':'Hill Dalit', 'Kami':'Hill Dalit', 'Sarki':'Hill Dalit', \
     'Bantar/Sardar':'Madhesi Dalit', 'Chamar/Harijan/Ram':'Madhesi Dalit', 'Chidimar':'Madhesi Dalit', \
     'Dhobi':'Madhesi Dalit', 'Dom':'Madhesi Dalit', 'Dusadh/Pasawan/Pasi':'Madhesi Dalit', \
     'Halkhor':'Madhesi Dalit', 'Khatwe':'Madhesi Dalit', 'Musahar':'Madhesi Dalit', 'Tatma/Tatwa':'Madhesi Dalit', \
     'Newar':'Newar', 'Sherpa':'Mtn/Hill Janajati A', 'Yakkha':'Mtn/Hill Janajati A', 'Jirel':'Mtn/Hill Janajati A', \
     'Kulung':'Mtn/Hill Janajati A', 'Yamphu':'Mtn/Hill Janajati A', 'Mewahang':'Mtn/Hill Janajati A', \
     'Gharti/Bhujel':'Mtn/Hill Janajati A', 'Khaling':'Mtn/Hill Janajati A', 'Darai':'Mtn/Hill Janajati A', \
     'Magar':'Mtn/Hill Janajati A', 'Chhantyal/Chhantel':'Mtn/Hill Janajati A', 'Aathpariya':'Mtn/Hill Janajati A', \
     'Bahing':'Mtn/Hill Janajati A', 'Rai':'Mtn/Hill Janajati A', 'Thulung':'Mtn/Hill Janajati A', \
     'Gurung':'Mtn/Hill Janajati A', 'Limbu':'Mtn/Hill Janajati A', 'Lepcha':'Mtn/Hill Janajati A', \
     'Sampang':'Mtn/Hill Janajati A', 'Dura':'Mtn/Hill Janajati A', 'Chamling':'Mtn/Hill Janajati A', \
     'Bantaba':'Mtn/Hill Janajati A', 'Loharung':'Mtn/Hill Janajati A', 'Thakali':'Mtn/Hill Janajati A', \
     'Bhote':'Mtn/Hill Janajati B', 'Bote':'Mtn/Hill Janajati B', 'Brahmu/Baramo':'Mtn/Hill Janajati B', \
     'Byasi/Sauka':'Mtn/Hill Janajati B', 'Chepang/Praja':'Mtn/Hill Janajati B', 'Danuwar':'Mtn/Hill Janajati B', \
     'Dolpo':'Mtn/Hill Janajati B', 'Ghale':'Mtn/Hill Janajati B', 'Hayu':'Mtn/Hill Janajati B', 'Hyolmo':'Mtn/Hill Janajati B', \
     'Kumal':'Mtn/Hill Janajati B', 'Kusunda':'Mtn/Hill Janajati B', 'Lhomi':'Mtn/Hill Janajati B', \
     'Lhopa':'Mtn/Hill Janajati B', 'Majhi':'Mtn/Hill Janajati B', 'Nachhiring':'Mtn/Hill Janajati B', \
     'Pahari':'Mtn/Hill Janajati B', 'Raji':'Mtn/Hill Janajati B', 'Raute':'Mtn/Hill Janajati B', \
     'Sunuwar':'Mtn/Hill Janajati B', 'Tamang':'Mtn/Hill Janajati B', 'Thami':'Mtn/Hill Janajati B', \
     'Topkegola':'Mtn/Hill Janajati B', 'Walung':'Mtn/Hill Janajati B', 'Dhanuk':'Tarai Janajati', \
     'Dhimal':'Tarai Janajati', 'Gangai':'Tarai Janajati', 'Jhangad/Dhagar':'Tarai Janajati', 'Kisan':'Tarai Janajati', \
     'Koche':'Tarai Janajati', 'Meche':'Tarai Janajati', 'Munda':'Tarai Janajati', 'Pattharkatta/Kushwadiya':'Tarai Janajati', \
     'Rajbansi':'Tarai Janajati', 'Satar/Santhal':'Tarai Janajati', 'Tajpuriya':'Tarai Janajati', 'Tharu':'Tarai Janajati', \
     'Musalman':'Musalman', 'Marwadi':'Other (MPB)', 'Punjabi/Shikh':'Other (MPB)', 'Bangali':'Other (MPB)', \
     'Dalit Others':'Other undefined', 'Janajati Others':'Other undefined', 'Undefined Others':'Other undefined', \
     'Terai Others':'Other undefined', 'Others':'Other undefined', 'Foreigner':'Foreigner', 'Khawas':'Other undefined'}
caste['social_groups'] = caste['Caste/Ethnicity'].map(sg)
grouped = pd.DataFrame()
grouped['district'] = caste['District']
grouped['unique'] = caste['social_groups']
grouped['number'] = caste['In number']
grouped = pd.DataFrame(grouped['number'].groupby([grouped['district'], grouped['unique']]).sum())
grouped = grouped.reset_index()
grouped = grouped.pivot(index='district', columns='unique', values='number')
grouped = grouped.reset_index()
grouped = grouped.fillna(0)
grouped['Total Pop Census'] = grouped.sum(axis=1)
grouped.iloc[:,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]] = \
   grouped.iloc[:,[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]].div(grouped['Total Pop Census'],axis=0)
grouped = grouped.set_index('district')
grouped.index = grouped.index.rename('District ')
grouped.to_csv('Data/caste_grouped.csv')

## Poverty

In [6]:
poverty = df['poverty1'][df['poverty1']['Year AD'] == 2011]
series = []
for district in poverty['District'].unique():
    indicators = poverty[(poverty['District'] == district)].groupby('Indicators').sum()['Value']
    series.append(indicators.rename(district).T)

df['poverty1T'] = pd.DataFrame(series)
df['poverty1T'].to_csv('Data/poverty1T.csv')

series = []
poverty = df['poverty2']

for district in poverty['District'].unique():
    subgroups = poverty[(poverty['District'] == district)].groupby('Sub Group').sum()['Value']
    series.append(subgroups.rename(district).T)

df['poverty2T'] = pd.DataFrame(series)
df['poverty2T'].to_csv('Data/poverty2T.csv')

## vaccinationsT

In [7]:
vacc = df['vaccinations'][df['vaccinations']['Year AD'] == '2011/12']
series = []

for district in vacc['District'].unique():
    vaccines = vacc[vacc['District'] == district].groupby('Vaccines').sum()
    vaccines.columns = [district]
    series.append(vaccines.T.ix[0])
    
df['vaccinationsT'] = pd.DataFrame(series)
df['vaccinationsT'].to_csv('Data/vaccinationsT.csv')

## watersanitationT

In [8]:
df['watersanitaion']
series = []

for district in df['watersanitaion']['Districts'].unique():
    sani = df['watersanitaion'][df['watersanitaion']['Districts'] == district].groupby('Sub group').sum()
    sani.columns = [district]
    series.append(sani.T.ix[0])
df['watersanitationT'] = pd.DataFrame(series)
df['watersanitationT'].to_csv('Data/watersanitationT.csv')

## mentalhealthT

In [9]:
mh = df['mentalhealth']
series = []

for district in mh['District'].unique():
    mental = mh[mh['District'] == district].groupby('Mental problem').sum()
    mental.columns = [district]
    series.append(mental.T.ix[0])

df['mentalhealthT'] = pd.DataFrame(series)
df['mentalhealthT'].to_csv('Data/mentalhealthT.csv')

## Communicable Disease

In [10]:
cd1 = df['communicabledisease1'][df['communicabledisease1']['Year AD'] == '2011/12']
series = []

for district in cd1['District'].unique():
    disease1 = cd1[cd1['District'] == district].groupby('Communicable Diseases').sum()
    disease1.columns = [district]
    series.append(disease1.T.ix[0])

df['communicabledisease1T'] = pd.DataFrame(series)
df['communicabledisease1T'].to_csv('Data/communicabledisease1T.csv')

cd2 = df['communicabledisease2'][df['communicabledisease2']['Year AD'] == '2011/12']
series = []

for district in cd2['District'].unique():
    disease2 = cd2[cd2['District'] == district].groupby('Diseases').sum()
    disease2.columns = [district]
    series.append(disease2.T.ix[0])

df['communicabledisease2T'] = pd.DataFrame(series)
df['communicabledisease2T'].to_csv('Data/communicabledisease2T.csv')

In [11]:
df['communicabledisease2T'].head()

Diseases,Acute Flacid Paralysis,Chicken pox,Diptheria,Hepatatis-B,Measles,Mumps,Neonatal Tetanus,Rubella,Tetanus,Tuberculosis,Whooping cough
Taplejung,8,218,0,56,32,463,0,9,0,28,2
Panchthar,0,113,0,8,10,492,1,0,0,24,11
Ilam,0,162,0,2,41,677,0,17,2,51,0
Jhapa,138,626,0,18,32,1546,0,20,0,1207,3
Morang,89,706,0,134,96,1205,0,36,2,849,4


In [12]:
df['hdi2011'].head()

,District,Zone,Geographical Region,Development Region,HDI
0,Achham,Seti,Hill,Far-Western,0.378
1,Arghakhanchi,Lumbini,Hill,Western,0.482
2,Banke,Bheri,Terai,Mid-Western,0.475
3,Baglung,Dhawalagiri,Hill,Western,0.478
4,Baitadi,Mahakali,Hill,Far-Western,0.416


In [13]:
print sorted(df.keys())

['caste', 'caste_grouped', 'caste_percent', 'communicabledisease1', 'communicabledisease1T', 'communicabledisease2', 'communicabledisease2T', 'hdi2011', 'master', 'mentalhealth', 'mentalhealthT', 'poverty1', 'poverty1T', 'poverty2', 'poverty2T', 'sexratio', 'vaccinations', 'vaccinationsT', 'watersanitaion', 'watersanitationT']


In [14]:
df['hdi2011'] = df['hdi2011'].set_index(['District '])
df['sexratio'] = df['sexratio'].set_index('District ')

pd.concat([grouped, df['poverty1T'], df['poverty2T'],
           df['vaccinationsT'], df['watersanitationT'], df['hdi2011'],
           df['sexratio'], df['communicabledisease1T'], df['communicabledisease2T'],
           df['mentalhealthT'],], axis=1).to_csv('Data/master.csv')

In [15]:
test = pd.read_csv('Data/master.csv')

In [16]:
test.head()

,Unnamed: 0,Foreigner,Hill Brahman,Hill Chhetree,Hill Dalit,Madhesi Brahman/Chhetree,Madhesi Dalit,Madhesi Other - OC A,Madhesi Other - OC B,Mtn/Hill Janajati A,...,Tetanus,Tuberculosis,Whooping cough,Alcoholism,Anxiety (Neurosis,Conversive disorder (Hysteria),Depression,Epilesy,Mental retardation,Psychosis
0,Achham,0.000000,0.102036,0.581749,0.135158,0.000101,0.000000,0.004921,0.0,0.006921,...,0.0,62.0,16.0,189.0,2202.0,281.0,1925.0,1118.0,69.0,413.0
1,Arghakhanchi,0.000000,0.327665,0.193248,0.209020,0.001604,0.000066,0.001058,0.0,0.193349,...,0.0,47.0,0.0,206.0,1597.0,112.0,1544.0,124.0,78.0,137.0
2,Baglung,0.000045,0.195385,0.205601,0.263230,0.001165,0.000000,0.000927,0.0,0.308176,...,4.0,40.0,0.0,600.0,3769.0,221.0,1695.0,386.0,349.0,429.0
3,Baitadi,0.000052,0.185498,0.603899,0.083153,0.001610,0.000128,0.052499,0.0,0.003870,...,6.0,141.0,1.0,125.0,407.0,45.0,518.0,29.0,76.0,39.0
4,Bajhang,0.000077,0.101953,0.726490,0.113103,0.000497,0.000102,0.000277,0.0,0.002055,...,1.0,90.0,4.0,162.0,1312.0,196.0,1178.0,253.0,44.0,178.0
